 # Functions to create and save graphs

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain

In [ ]:
def save_ttest_plots(input_folder, 
                     output_folder, 
                     file_prefix="t_vals_", 
                     byte_pos_range=range(0, 32), 
                     byte_value_range=range(0x00, 0x100),
                     word_indices=None):
    """
    Save t-test value plots as PNG files for specified byte positions and values.

    NOTE: word_indices was added ad-hoc to allow for plotting of t-test values for specific S1 words, when additional LA of S1[0] and S1[12] was done.

    Parameters:
        input_folder (str): Path to the folder containing t-test value files.
        output_folder (str): Path to the folder where plots will be saved.
        file_prefix (str): Prefix for the t-test value files (default: "t_vals_").
        byte_pos_range (range): Range of byte positions to process (default: 0-31).
        byte_value_range (range): Range of byte values to process (default: 0x00-0xff).
        word_indices (list): List of word indices to process (default: None). If None, only byte positions are processed.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Generate a colormap for the byte values
    num_colors = len(byte_value_range)
    colors = plt.cm.get_cmap('hsv', num_colors)

    # Iterate over byte positions
    if word_indices is None:
        word_indices = [-1]  # Default to processing all byte positions if not specified

    for word_pos in word_indices:
        for byte_pos in byte_pos_range:
            t_values_array_np = []

            # Load the t-test value files for each byte value
            for byte_value in byte_value_range:
                byte_value_hex = f"{byte_value:02x}"
                t_values_file = None
                if word_pos == -1:
                    t_values_file = os.path.join(input_folder, f"{file_prefix}{byte_pos}_{byte_value_hex}.npy")
                else:
                    t_values_file = os.path.join(input_folder, f"{file_prefix}{word_pos}_{byte_pos}_{byte_value_hex}.npy")
                if not os.path.exists(t_values_file):
                    print(f"File not found: {t_values_file}")
                    continue

                t_values = np.load(t_values_file)
                t_values_array_np.append(t_values)

            # Convert the list to a NumPy array for easier indexing
            t_values_array_np = np.array(t_values_array_np)

            # Create a figure for the plot
            plt.figure(figsize=(10, 5))

            # Plot t-values for each byte value
            for y, byte_value in enumerate(byte_value_range):
                if y < len(t_values_array_np):  # Ensure the index is valid
                    plt.scatter(range(len(t_values_array_np[y])), t_values_array_np[y], color=colors(y), s=10, alpha=0.5)

            # Add threshold lines
            plt.axhline(y=4.5, color='r', linestyle='--', label='Threshold 4.5')
            plt.axhline(y=-4.5, color='r', linestyle='--', label='Threshold -4.5')

            # Set labels and title
            plt.xlabel('Trace Sample Point')
            plt.ylabel('t-value')

            # Save the plot as a PNG file
            output_file = None
            if word_pos == -1:
                output_file = os.path.join(output_folder, f"{file_prefix}{byte_pos}.png")
            else:
                output_file = os.path.join(output_folder, f"{file_prefix}{word_pos}_{byte_pos}.png")
            plt.savefig(output_file, bbox_inches='tight', dpi=600)
            print(f"Plot saved to {output_file}")

            # Close the plot to free memory
            plt.close()

# Plot creation for all specific t-tests

In [ ]:
# STM - Key 1
input_folder = "_LA-STM-1"
output_folder = "_LA-STM-1/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(0, 32), byte_value_range=range(0x100))

In [ ]:
# STM - Key 2
input_folder = "_LA-STM-2"
output_folder = "_LA-STM-2/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(0, 32), byte_value_range=range(0x100))

In [ ]:
# XMEGA - Key 1
input_folder = "_LA-XMEGA-1"
output_folder = "_LA-XMEGA-1/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(0, 32), byte_value_range=range(0x100))

In [ ]:
# XMEGA - Key 2
input_folder = "_LA-XMEGA-2"
output_folder = "_LA-XMEGA-2/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(0, 32), byte_value_range=range(0x100))

### Additional Plot saving after LA of S1[0] and S1[12]

In [ ]:
# STM - Key 1
input_folder = "_LA-STM-1-ADDITIONAL-S1-WORDS"
output_folder = "_LA-STM-1-ADDITIONAL-S1-WORDS/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(4), byte_value_range=range(0x100), word_indices=[0, 12])

In [ ]:
# STM - Key 2
input_folder = "_LA-STM-2-ADDITIONAL-S1-WORDS"
output_folder = "_LA-STM-2-ADDITIONAL-S1-WORDS/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(4), byte_value_range=range(0x100), word_indices=[0, 12])

In [ ]:
# STM - Key 2
input_folder = "_LA-XMEGA-1-ADDITIONAL-S1-WORDS"
output_folder = "_LA-XMEGA-1-ADDITIONAL-S1-WORDS/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(4), byte_value_range=range(0x100), word_indices=[0, 12])

In [ ]:
# STM - Key 2
input_folder = "_LA-XMEGA-2-ADDITIONAL-S1-WORDS"
output_folder = "_LA-XMEGA-2-ADDITIONAL-S1-WORDS/plots"
save_ttest_plots(input_folder, output_folder, file_prefix="t_stats_", byte_pos_range=range(4), byte_value_range=range(0x100), word_indices=[0, 12])

# Custom functions

In [ ]:
def plot_ttest_with_trace(ttest_folder, traces_folder, output_folder, byte_pos, byte_value_range, trace_file, trace_len):
    """
    Plot t-test values for a range of byte values at a specific byte position, with a single trace in the background.

    Parameters:
        ttest_folder (str): Path to the folder containing t-test value files.
        traces_folder (str): Path to the folder containing trace files.
        output_folder (str): Path to the folder where the plot will be saved.
        byte_pos (int): Byte position to plot.
        byte_value_range (range): Range of byte values to plot (e.g., range(0x00, 0x10)).
        trace_file (str): Name of the trace file to load.
        trace_index (int): Index of the trace to overlay (default: 0).
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Load the first trace directly from the file
    trace_path = os.path.join(traces_folder, trace_file)
    if not os.path.exists(trace_path):
        print(f"Trace file not found: {trace_path}")
        return

    # Read only the first trace
    with open(trace_path, 'rb') as f:
        trace = np.frombuffer(f.read(trace_len * np.dtype(np.uint16).itemsize), dtype=np.uint16)

    # Create a figure and two y-axes
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot the trace on the primary y-axis
    ax1.plot(trace, 'k-', alpha=0.5, label="Trace (background)")
    ax1.set_xlabel("Sample Points")
    ax1.set_ylabel("Amplitude", color='black')
    ax1.tick_params(axis='y', labelcolor='black')

    # Create a secondary y-axis for the t-test values
    ax2 = ax1.twinx()

    # Plot t-test values for each byte value in the range
    for byte_value in byte_value_range:
        ttest_file = os.path.join(ttest_folder, f"t_stats_{byte_pos}_{byte_value:02x}.npy")
        if not os.path.exists(ttest_file):
            print(f"T-test file not found: {ttest_file}")
            continue

        ttest_values = np.load(ttest_file)
        ax2.plot(ttest_values)

    ax2.set_ylabel("T-value", color='red')
    ax2.tick_params(axis='y', labelcolor='red')

    # Add threshold lines on the secondary y-axis
    ax2.axhline(y=4.5, color='r', linestyle='--', label='Threshold 4.5')
    ax2.axhline(y=-4.5, color='r', linestyle='--', label='Threshold -4.5')

    # Add a legend
    fig.legend(loc="upper right", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)

    # Save the plot as a PNG file
    output_file = os.path.join(output_folder, f"t_test_overlay_{byte_pos}.png")
    plt.savefig(output_file, bbox_inches='tight', dpi=600)
    print(f"Plot saved to {output_file}")

    # Show the plot
    plt.show()

In [ ]:
# Example parameters
ttest_folder = "_LA-XMEGA-1"
traces_folder = "ChaCha-100-000-Random-Nonce-XMEGA"
output_folder = "_LA-XMEGA-1/plots"
byte_pos = 4
byte_value_range = range(0x00, 0x3)  # Plot for byte values 0x00 to 0x0F
trace_file = "traces_random_all.bin"
trace_index = 0
trace_len = 21304

# Call the function
plot_ttest_with_trace(ttest_folder, traces_folder, output_folder, byte_pos, byte_value_range, trace_file, trace_len)

In [ ]:
def plot_ttest(ttest_folder, byte_pos, byte_value_range, trace_len):
    """
    Plot t-test values.

    Parameters:
        ttest_folder (str): Path to the folder containing t-test value files.
        byte_pos (int): Byte position to plot.
        byte_value_range (range): Range of byte values to plot (e.g., range(0x00, 0x10)).
        trace_len (int): Trace lenght / samples count.
    """
    # Create a figure for the plot
    plt.figure(figsize=(12, 6))

    # Generate a colormap for the byte values
    num_colors = len(byte_value_range)
    colors = plt.cm.get_cmap('hsv', num_colors)

    # Plot t-test values for each byte value in the range
    for y, byte_value in enumerate(byte_value_range):
        ttest_file = os.path.join(ttest_folder, f"t_stats_{byte_pos}_{byte_value:02x}.npy")
        if not os.path.exists(ttest_file):
            print(f"T-test file not found: {ttest_file}")
            continue

        ttest_values = np.load(ttest_file)
#         plt.plot(ttest_values, color=colors(y), label=f"Byte Value {byte_value:02x}")
        plt.scatter(range(len(ttest_values)), ttest_values, color=colors(y), s=10, alpha=0.5)

    # Add threshold lines
    plt.axhline(y=4.5, color='r', linestyle='--', label='Threshold 4.5')
    plt.axhline(y=-4.5, color='r', linestyle='--', label='Threshold -4.5')

    # Set labels and title
    plt.xlabel('Trace Sample Point')
    plt.ylabel('t-value')
    plt.title(f'T-Values for Byte Position {byte_pos}')
    plt.xlim(500,1000)
    

#     Show the plot
    plt.show()

In [ ]:
plot_ttest("_LA-STM-1", 4, range(256), 9168)